In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
import os
import cv2
import random

In [4]:
TRAIN_DIR = 'input/stage1_train/'
TEST_DIR = 'input/stage1_test/'
WIDTH = 128
HEIGHT = 128
CHANNEL = 3

In [5]:
train_ids = os.listdir('input/stage1_train')

In [6]:
X_train = []
Y_train = []
for d in train_ids:
    base_path = os.path.join(TRAIN_DIR, d)
    img = cv2.imread(os.path.join(base_path,"images",d+".png"))
    img = cv2.resize(img,(HEIGHT,WIDTH))/255
    X_train.append(img)
    
    masks_list = os.listdir(os.path.join(base_path,"masks"))
    masks = []
    for m in masks_list:
        mask = cv2.imread(os.path.join(base_path,"masks",m), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,(HEIGHT,WIDTH))/255
        masks.append(mask)
    masks = np.array(masks)
    masks = np.amax(masks,axis=0)
    masks = np.expand_dims(masks, axis=-1)
    Y_train.append(masks)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [8]:
from scipy.ndimage import rotate as rot
def augment(x,y):
    x_mirror_lr = []
    x_mirror_ud = []
    x_rotate = []
    x_rotate_2 = []
    x_rotate_3 = []
    x_rotate_4 = []
    y_mirror_lr = []
    y_mirror_ud = []
    y_rotate = []
    y_rotate_2 = []
    y_rotate_3 = []
    y_rotate_4 = []
    for i in range(0,x.shape[0]):
        band_1 = x[i,:,:,0]
        band_2 = x[i,:,:,1]
        band_3 = x[i,:,:,2]
        band_y = y[i,:,:,0]
            
        # mirror left-right
        band_1_mirror_lr = np.flip(band_1, 0)
        band_2_mirror_lr = np.flip(band_2, 0)
        band_3_mirror_lr = np.flip(band_3, 0)
        x_mirror_lr.append(np.dstack((band_1_mirror_lr, band_2_mirror_lr, band_3_mirror_lr)))
        y_mirror_lr.append(np.expand_dims(np.flip(band_y, 0), axis=-1))
        
        # mirror up-down
        band_1_mirror_ud = np.flip(band_1, 1)
        band_2_mirror_ud = np.flip(band_2, 1)
        band_3_mirror_ud = np.flip(band_3, 1)
        x_mirror_ud.append(np.dstack((band_1_mirror_ud, band_2_mirror_ud, band_3_mirror_ud)))
        y_mirror_ud.append(np.expand_dims(np.flip(band_y, 1), axis=-1))
        
        #rotate 
        band_1_rotate = rot(band_1, 45, reshape=False)
        band_2_rotate = rot(band_2, 45, reshape=False)
        band_3_rotate = rot(band_3, 45, reshape=False)
        x_rotate.append(np.dstack((band_1_rotate, band_2_rotate, band_3_rotate)))
        y_rotate.append(np.expand_dims(rot(band_y, 45, reshape=False), axis=-1))
        
        #rotate2 
        band_1_rotate_2 = rot(band_1, -45, reshape=False)
        band_2_rotate_2 = rot(band_2, -45, reshape=False)
        band_3_rotate_2 = rot(band_3, -45, reshape=False)
        x_rotate_2.append(np.dstack((band_1_rotate_2, band_2_rotate_2, band_3_rotate_2)))
        y_rotate_2.append(np.expand_dims(rot(band_y, -45, reshape=False), axis=-1))
        
        #rotate3 
        band_1_rotate_3 = rot(band_1, 135, reshape=False)
        band_2_rotate_3 = rot(band_2, 135, reshape=False)
        band_3_rotate_3 = rot(band_3, 135, reshape=False)
        x_rotate_3.append(np.dstack((band_1_rotate_3, band_2_rotate_3, band_3_rotate_3)))
        y_rotate_3.append(np.expand_dims(rot(band_y, 135, reshape=False), axis=-1))
        
        #rotate4 
        band_1_rotate_4 = rot(band_1, -135, reshape=False)
        band_2_rotate_4 = rot(band_2, -135, reshape=False)
        band_3_rotate_4 = rot(band_3, -135, reshape=False)
        x_rotate_4.append(np.dstack((band_1_rotate_4, band_2_rotate_4, band_3_rotate_4)))
        y_rotate_4.append(np.expand_dims(rot(band_y, -135, reshape=False), axis=-1))
        
    x = np.concatenate((x, np.array(x_mirror_lr), np.array(x_mirror_ud), np.array(x_rotate), np.array(x_rotate_2), np.array(x_rotate_3), np.array(x_rotate_4)))
    y = np.concatenate((y, np.array(y_mirror_lr), np.array(y_mirror_ud), np.array(y_rotate), np.array(y_rotate_2), np.array(y_rotate_3), np.array(y_rotate_4)))
    
    del x_mirror_lr,x_mirror_ud,x_rotate,x_rotate_2,x_rotate_3,x_rotate_4
    del y_mirror_lr,y_mirror_ud,y_rotate,y_rotate_2,y_rotate_3,y_rotate_4
    
    return x,y

In [9]:
X_train,Y_train = augment(X_train,Y_train)
X_train.shape, Y_train.shape

((4690, 128, 128, 3), (4690, 128, 128, 1))

In [10]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [11]:
import tensorflow as tf

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [12]:
def create_model():
    inputs = Input((HEIGHT, WIDTH, CHANNEL))
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
    
    return model

In [13]:
model = create_model()
#model.summary()

In [15]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model_02.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=50, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

Train on 4221 samples, validate on 469 samples
Epoch 1/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.1969 - mean_iou: 0.5220 - val_loss: 0.1260 - val_mean_iou: 0.5804
Epoch 2/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.1220 - mean_iou: 0.6132 - val_loss: 0.0975 - val_mean_iou: 0.6391
Epoch 3/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0970 - mean_iou: 0.6568 - val_loss: 0.0848 - val_mean_iou: 0.6713
Epoch 4/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0852 - mean_iou: 0.6818 - val_loss: 0.0741 - val_mean_iou: 0.6909
Epoch 5/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0801 - mean_iou: 0.6978 - val_loss: 0.0673 - val_mean_iou: 0.7041
Epoch 6/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0756 - mean_iou: 0.7090 - val_loss: 0.0648 - val_mean_iou: 0.7134
Epoch 7/50
4221/4221 [==============================] - 121s 29ms/step 

Epoch 28/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0567 - mean_iou: 0.7606 - val_loss: 0.0512 - val_mean_iou: 0.7610
Epoch 29/50
4221/4221 [==============================] - 120s 29ms/step - loss: 0.0562 - mean_iou: 0.7614 - val_loss: 0.0513 - val_mean_iou: 0.7617
Epoch 30/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0561 - mean_iou: 0.7620 - val_loss: 0.0507 - val_mean_iou: 0.7623
Epoch 31/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0561 - mean_iou: 0.7626 - val_loss: 0.0536 - val_mean_iou: 0.7629
Epoch 32/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0574 - mean_iou: 0.7632 - val_loss: 0.0499 - val_mean_iou: 0.7635
Epoch 33/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0560 - mean_iou: 0.7637 - val_loss: 0.0508 - val_mean_iou: 0.7640
Epoch 34/50
4221/4221 [==============================] - 121s 29ms/step - loss: 0.0558 - mean_iou: 0.7643 - val_